In [36]:
import pandas as pd
import numpy as np
import random

import tensorflow as tf

from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPool1D, Flatten, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.activations import relu, sigmoid
from tensorflow.keras.optimizers import Adam, SGD

from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [73]:
def configure_nn(input_size,
                layers=None, 
                dropout_rate=0, 
                filters=10,
                kernel_size=10,
                stride=10,
                pool_size=2,
                optimizer='Adam',
                loss='binary_crossentropy',
                 kernel_initializer='lecun_normal',
                 kernel_regularizer=tf.keras.regularizers.L2(0.01)
                ):
    """ Layers argument shape:
    [[number of nodes, activate function], 
    [number of nodes, activate function],
    ...]
    
    """
        
    model = Sequential()
    model.add(Embedding(10000, 10, input_length=15))
    model.add(Conv1D(filters=filters, kernel_size=kernel_size,
                    strides=10, padding='same', activation='relu'))
    
    model.add(MaxPool1D(pool_size=pool_size))
    model.add(Flatten())
    
    if dropout_rate > 0:
        model.add(Dropout(dropout_rate))
    
    if layers != None:
        for node in layers[1:]:
            model.add(Dense(node[0], activation=node[1], kernel_initializer=kernel_initializer, 
                            kernel_regularizer=kernel_regularizer))
    model.add(Dense(10, activation='sigmoid'))
    model.add(Dense(1, activation='sigmoid', kernel_initializer=kernel_initializer, 
                    kernel_regularizer=kernel_regularizer))
    
    model.compile(loss=loss, 
                 optimizer=optimizer,
                 metrics=['accuracy'])
    
    return model

In [76]:
model = configure_nn(15)

In [77]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 15, 10)            100000    
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 2, 10)             1010      
_________________________________________________________________
max_pooling1d_10 (MaxPooling (None, 1, 10)             0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 10)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 11        
Total params: 101,131
Trainable params: 101,131
Non-trainable params: 0
_______________________________________________

In [58]:
data = pd.read_csv('../tokenized_data.csv')

In [59]:
data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,sentiment
0,0,0,0,0,0,1189,369,3135,38,325,82,1037,553,2424,9704,1
1,0,0,0,0,0,0,0,22,293,3,12,63,1,810,561,1
2,0,0,0,0,0,0,0,0,23,313,3,13,333,774,6227,0
3,0,0,0,0,0,0,0,0,0,0,0,379,293,1167,3643,1
4,0,0,0,0,0,0,0,0,0,0,8,72,18,4949,687,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
159996,0,0,0,0,1,2178,27,794,263,270,9,36,52,3523,624,0
159997,0,0,0,0,0,0,0,0,901,297,137,318,5,781,31,0
159998,0,1251,475,1010,1626,2231,348,1051,558,202,98,339,4,6843,50,0


In [61]:
features = data.columns.tolist()
features.remove('sentiment')

In [62]:
X_train, X_test, y_train, y_test = train_test_split(data[features], data['sentiment'], test_size=0.33, random_state=42)

In [78]:
model.fit(X_train, y_train, epochs=100, verbose=1)

Epoch 1/100
3350/3350 [==============================] - 7s 2ms/step - loss: 0.7093 - accuracy: 0.5011
Epoch 2/100
3350/3350 [==============================] - 7s 2ms/step - loss: 0.6910 - accuracy: 0.5338
Epoch 3/100
3350/3350 [==============================] - 7s 2ms/step - loss: 0.6368 - accuracy: 0.6588
Epoch 4/100
3350/3350 [==============================] - 11s 3ms/step - loss: 0.5659 - accuracy: 0.7256
Epoch 5/100
3350/3350 [==============================] - 13s 4ms/step - loss: 0.5203 - accuracy: 0.7620
Epoch 6/100
3350/3350 [==============================] - 11s 3ms/step - loss: 0.4844 - accuracy: 0.7889
Epoch 7/100
3350/3350 [==============================] - 11s 3ms/step - loss: 0.4580 - accuracy: 0.8069
Epoch 8/100
3350/3350 [==============================] - 15s 4ms/step - loss: 0.4387 - accuracy: 0.8180
Epoch 9/100
3350/3350 [==============================] - 13s 4ms/step - loss: 0.4203 - accuracy: 0.8306
Epoch 10/100
3350/3350 [==============================] - 11s 3ms/s

KeyboardInterrupt: 